# What-if scenario evaluation for carbon sequestration

This notebook uses the [COMET-Farm API](https://gitlab.com/comet-api/api-docs/-/tree/master/) to derive soil carbon sequestration information for agricultural fields.

### Micromamba environment setup

To install the required packages, please refer to our [documentation](https://github.com/microsoft/farmvibes-ai/tree/main/notebooks).

### Required Input

FarmVibes.AI relies on [`SeasonalFieldInformation`](https://github.com/microsoft/farmvibes-ai/tree/main/src/vibe_core/vibe_core/data/farm.py) objects to represent farm practices for a field in a given season. It 
contains information related to harvest, planting, tillage, organic amendments, and fetilization applications. These objects contain information related to harvest, planting, tillage, organic amendments, and fertilization applications. In order to estimate the amount of soil carbon sequestered by a given farm, we need to provide two lists of `SeasonalFieldInformation` objects:

 * `baseline_seasonal_fields: List[SeasonalFieldInformation]`: Containing information about past agriculture activities on a given management field. Each `SeasonalFieldInformation` object that represents one year of crop information and it is recommended to use **ten calendar years** to represent the baseline carbon information.
 * `scenario_seasonal_fields: List[SeasonalFieldInformation]`: Containing information about agriculture activities to be performed for at least **two calendar years** following seasons after `baseline_seasonal_fields`. Similar to the baseline_seasonal_fields, each `SeasonalFieldInformation` object stores farming information for each crop year.


The objects are stored in files  [`baseline_seasonal_fields.json`](./baseline_seasonal_fields.json) and [`scenario_seasonal_fields.json`](./scenario_seasonal_fields.json) in JSON format and are loaded in memory in the next cells.

### Import required packages

In [ ]:
import json
from datetime import datetime
from typing import Any, Dict

from vibe_core.data import (
    FertilizerInformation,
    HarvestInformation,
    OrganicAmendmentInformation,
    SeasonalFieldInformation,
    TillageInformation,
)

In [ ]:
def seasonal_field_from_dict(data: Dict[str, Any]) -> SeasonalFieldInformation:
    harvests = [HarvestInformation(**h) for h in data["harvests"]]
    tillages = [TillageInformation(**t) for t in data["tillages"]]
    organic_amendments = [OrganicAmendmentInformation(**o) for o in data["organic_amendments"]]
    fertilizers = [FertilizerInformation(**f) for f in data["fertilizers"]]

    return SeasonalFieldInformation(
        id=data["id"],
        time_range=(
            datetime.fromisoformat(data["time_range"][0]),
            datetime.fromisoformat(data["time_range"][1]),
        ),
        geometry=data["geometry"],
        assets=data["assets"],
        crop_name=data["crop_name"],
        crop_type=data["crop_type"],
        properties=data["properties"],
        fertilizers=fertilizers,
        harvests=harvests,
        tillages=tillages,
        organic_amendments=organic_amendments,
    )


### Load FarmVibes.AI SeasonFieldInformationObjects objects

The next cell loads two lists of seasonal field objects stored in files [`baseline_seasonal_fields.json`](./baseline_seasonal_fields.json) and [`scenario_seasonal_fields.json`](./scenario_seasonal_fields.json) that are required for soil carbon sequestration estimation.

In [ ]:
baseline_fields_file = "./baseline_seasonal_fields.json"
with open(baseline_fields_file) as json_file:
    baseline_seasonal_fields = [
        seasonal_field_from_dict(seasonal_field_dict)
        for seasonal_field_dict in json.load(json_file)
    ]

scenario_fields_file = "./scenario_seasonal_fields.json"
with open(scenario_fields_file) as json_file:
    scenario_seasonal_fields = [
        seasonal_field_from_dict(seasonal_field_dict)
        for seasonal_field_dict in json.load(json_file)

]

## Details about the output of the COMET-Farm API

1. All greenhouse gas models in the COMET-Farm platform (Example: DayCent, rice
   methane, residue burning, liming, urea fertilizer, etc.) are run against the
   baseline scenario and then against each conservation scenario, on each unique
   combination of soil map units found within each parcel or point for each model
   run.

2.	Aggregated Baseline and Scenario results totals for all models are also
    returned, named "Baseline" and "Scenario".

An example output follows:

```json
{
    "@name": "scenario: 21/07/2022 10:34:05",
    "Carbon": {
        "SoilCarbon": "-1679.9",
        "BiomassBurningCarbon": "0",
        "SoilCarbonStock2000": "5511.312",
        "SoilCarbonStockBegin": "5753.6314",
        "SoilCarbonStockEnd": "5759.8725"
    },
    "CO2": {
        "LimingCO2": "0",
        "UreaFertilizationCO2": "8.5587",
        "DrainedOrganicSoilsCO2": "0"
    },
    "N2O": {
        "SoilN2O": "536.1286",
        "SoilN2O_Direct": "451.9349",
        "SoilN2O_Indirect_Volatilization": "84.1937",
        "SoilN2O_Indirect_Leaching": "0",
        "WetlandRiceCultivationN2O": "0",
        "BiomassBurningN2O": "0",
        "DrainedOrganicSoilsN2O": "0"
    },
    "CH4": {
        "SoilCH4": "0",
        "WetlandRiceCultivationCH4": "0",
        "BiomassBurningCH4": "0"
    }
}
```

## Pre-requisites to run this notebook

1. Sign up with https://comet-farm.com/. The email registered there will be used when there are error messages, or when a request fails.
2. Sign up with https://dashboard.ngrok.com/ to allow the TerraVibes carbon endpoint to be accessible by COMET-Farm API's webhooks.
   1. Navigate to "Getting Started"/"Your Auth token" and copy the Auth token
   2. Update copied auth token in variable "NGROK_AUTH_TOKEN" in next cell

In [ ]:
COMET_REGISTERED_EMAIL = ""
NGROK_AUTH_TOKEN = ""

In [ ]:

from vibe_core.client import FarmvibesAiClient, get_default_vibe_client

### On running this workflow

*Note*: Running this workflow will expose an endpoint within the FarmVibes.AI worker container publicly. This connection will exist for as long as the workflow is running, and will be closed as soon as the workflow completes.

This is a required step for receiving results from the COMET-Farm API. Incorrect ngrok configuration will cause the workflow to fail.

In [ ]:
client: FarmvibesAiClient = get_default_vibe_client()
WORKFLOW_NAME = "farm_ai/carbon_local/carbon_whatif"

In [ ]:
client.document_workflow(WORKFLOW_NAME)

In [ ]:
parameters = {
  "ngrok_token": NGROK_AUTH_TOKEN,
  "comet_support_email": COMET_REGISTERED_EMAIL
}

run = client.run(
    WORKFLOW_NAME,
    name="whatif_carbon_seq",
    input_data={
        "baseline_seasonal_fields": baseline_seasonal_fields,
        "scenario_seasonal_fields": scenario_seasonal_fields,
    },
    parameters=parameters
)
run.monitor()

Use the cells below to inspect the running workflow.

Depending on the COMET-Farm API resources availability, workflows will take longer to complete.

If the workflow does not complete within 2 (two) hours and the workflow is still running, use the COMET-Farm support at appnrel@colostate.edu. On failure, error information will be sent to the email registered with COMET.

In [ ]:
carbon_offset_info = run.output["carbon_output"][0]
print(f"Estimated carbon offset is {carbon_offset_info.carbon}")